# Задание: Табличная функция для поденных сумм платежей

## Цель задания
Необходимо написать табличную функцию SQL, которая будет возвращать по ClientId и интервалу дат (тип Date) поденные суммы платежей. Если за указанный день не было платежей, то функция должна возвращать 0. Интервалы дат могут охватывать несколько лет.

## Структура данных
Есть таблица платежей клиентов ClientPayments следующего вида:

```sql
ClientPayments
(
    Id bigint,          -- первичный ключ таблицы
    ClientId bigint,    -- Id клиента
    Dt datetime2(0),    -- дата платежа
    Amount money        -- сумма платежа
)
```

## Требования к функции
1. Функция должна принимать параметры:
   - `@ClientId` (bigint) - идентификатор клиента
   - `@Sd` (Date) - начальная дата интервала
   - `@Ed` (Date) - конечная дата интервала

2. Функция должна возвращать результат со следующими колонками:
   - `Dt` (Date) - дата
   - `Сумма` (money) - сумма платежей за день (0 если платежей не было)

3. Функция должна возвращать запись для каждого дня в указанном интервале включительно.

4. Если за день не было платежей, возвращать 0.

## Примеры работы

### Пример 1
**Входные данные:**
- ClientId = 1
- Sd = 2022-01-02
- Ed = 2022-01-07

**Ожидаемый результат:**
| Dt | Сумма |
|----|-------|
| 2022-01-02 | 0 |
| 2022-01-03 | 100 |
| 2022-01-04 | 0 |
| 2022-01-05 | 450 |
| 2022-01-06 | 0 |
| 2022-01-07 | 50 |

### Пример 2
**Входные данные:**
- ClientId = 2
- Sd = 2022-01-04
- Ed = 2022-01-11

**Ожидаемый результат:**
| Dt | Сумма |
|----|-------|
| 2022-01-04 | 0 |
| 2022-01-05 | 278 |
| 2022-01-06 | 0 |
| 2022-01-07 | 0 |
| 2022-01-08 | 0 |
| 2022-01-09 | 0 |
| 2022-01-10 | 300 |
| 2022-01-11 | 0 |


Решение

In [ ]:
-- =============================================
-- Табличная функция для получения поденных сумм платежей клиента
-- =============================================
CREATE OR ALTER FUNCTION dbo.GetDailyPayments
(
    @ClientId bigint,
    @Sd date,
    @Ed date
)
RETURNS TABLE
AS
RETURN
(
    WITH DateRange AS
    (
        -- Генерируем все даты в указанном интервале
        SELECT CAST(@Sd AS date) AS Dt
        
        UNION ALL
        
        SELECT DATEADD(DAY, 1, Dt) AS Dt
        FROM DateRange
        WHERE Dt < @Ed
    )
    SELECT 
        dr.Dt,
        ISNULL(SUM(cp.Amount), 0) AS Сумма
    FROM DateRange dr
    LEFT JOIN ClientPayments cp 
        ON cp.ClientId = @ClientId 
        AND CAST(cp.Dt AS date) = dr.Dt
    GROUP BY dr.Dt
    OPTION (MAXRECURSION 10000)  -- Ограничение для рекурсивного CTE (до ~27 лет)
);


## Реализация функции

Функция использует рекурсивное CTE (Common Table Expression) для генерации всех дат в указанном интервале, затем делает LEFT JOIN с таблицей платежей и группирует по дате, используя ISNULL для замены NULL на 0.

### Особенности реализации:
1. **DateRange CTE** - генерирует все даты от @Sd до @Ed включительно
2. **LEFT JOIN** - соединяет даты с платежами, чтобы получить все даты даже без платежей
3. **ISNULL(SUM(...), 0)** - заменяет NULL на 0 для дней без платежей
4. **MAXRECURSION 10000** - увеличивает лимит рекурсии для больших интервалов (до ~27 лет)


In [ ]:
-- Пример использования функции для создания тестовых данных и проверки

-- Создание тестовой таблицы (для демонстрации)
/*
CREATE TABLE ClientPayments
(
    Id bigint IDENTITY(1,1) PRIMARY KEY,
    ClientId bigint NOT NULL,
    Dt datetime2(0) NOT NULL,
    Amount money NOT NULL
);

-- Вставка тестовых данных
INSERT INTO ClientPayments (ClientId, Dt, Amount) VALUES
(1, '2022-01-03 10:00:00', 100.00),
(1, '2022-01-05 14:30:00', 450.00),
(1, '2022-01-07 09:15:00', 50.00),
(2, '2022-01-05 11:20:00', 278.00),
(2, '2022-01-10 16:45:00', 300.00);
*/


In [ ]:
-- Пример 1: ClientId = 1, период 2022-01-02 до 2022-01-07
SELECT * FROM dbo.GetDailyPayments(1, '2022-01-02', '2022-01-07')
ORDER BY Dt;


In [ ]:
-- Пример 2: ClientId = 2, период 2022-01-04 до 2022-01-11
SELECT * FROM dbo.GetDailyPayments(2, '2022-01-04', '2022-01-11')
ORDER BY Dt;


## Альтернативная реализация (без рекурсии)

Для очень больших интервалов или если рекурсия нежелательна, можно использовать другой подход с таблицей чисел или временной таблицей:

```sql
CREATE OR ALTER FUNCTION dbo.GetDailyPayments_v2
(
    @ClientId bigint,
    @Sd date,
    @Ed date
)
RETURNS TABLE
AS
RETURN
(
    SELECT 
        DATEADD(DAY, numbers.number, @Sd) AS Dt,
        ISNULL(SUM(cp.Amount), 0) AS Сумма
    FROM 
        (SELECT 0 AS number
         UNION ALL SELECT 1 UNION ALL SELECT 2 UNION ALL SELECT 3 UNION ALL SELECT 4
         UNION ALL SELECT 5 UNION ALL SELECT 6 UNION ALL SELECT 7 UNION ALL SELECT 8 UNION ALL SELECT 9
        ) AS base
        CROSS APPLY 
        (SELECT base.number AS number
         UNION ALL SELECT base.number + 10 UNION ALL SELECT base.number + 20 
         UNION ALL SELECT base.number + 30 UNION ALL SELECT base.number + 40
         UNION ALL SELECT base.number + 50 UNION ALL SELECT base.number + 60 
         UNION ALL SELECT base.number + 70 UNION ALL SELECT base.number + 80 UNION ALL SELECT base.number + 90
        ) AS tens
        CROSS APPLY
        (SELECT tens.number AS number
         UNION ALL SELECT tens.number + 100 UNION ALL SELECT tens.number + 200 
         UNION ALL SELECT tens.number + 300 UNION ALL SELECT tens.number + 400
         UNION ALL SELECT tens.number + 500 UNION ALL SELECT tens.number + 600 
         UNION ALL SELECT tens.number + 700 UNION ALL SELECT tens.number + 800 UNION ALL SELECT tens.number + 900
        ) AS numbers
    LEFT JOIN ClientPayments cp 
        ON cp.ClientId = @ClientId 
        AND CAST(cp.Dt AS date) = DATEADD(DAY, numbers.number, @Sd)
    WHERE DATEADD(DAY, numbers.number, @Sd) <= @Ed
    GROUP BY DATEADD(DAY, numbers.number, @Sd)
);
```

**Примечание:** Основная реализация с рекурсивным CTE проще и читабельнее для большинства случаев.
